In [1]:
"""В прошлом модуле мы совершили множество преобразований над нашей таблицей — давайте вспомним их:

удалили столбцы index и Coordinates;
создали признак средней площади одной комнаты MeanRoomsSquare и ввели коэффициент соотношения площади 
здания к площади участка — AreaRatio;
преобразовали признак даты продажи Date в формат datetime и создали на его основе следующие столбцы: 
номер месяц продажи (MonthSale), номер дня недели продажи (WeekdaySale), «признак-мигалку» выходного дня (Weekend);
заменили признак года постройки здания YearBuilt на его возраст AgeBuilding;
извлекли из признака адреса объекта Address новый признак подтипа улицы StreetType и удалили столбец с адресом;
уменьшили число уникальных наименований агентств по недвижимости (SellerG), а также число пригородов (Suburb);
выделили категориальные признаки и преобразовали их в тип данных сategory;
заменили сокращённые названия категорий признака типа объекта Type на их полные названия 
(h — house, t —  townhouse, u — unit)."""

#Прочитаем датасет и выведем первые пять его строк:
import pandas as pd

melb_df = pd.read_csv('data_test/melb_data_fe.csv')
melb_df.head()
#Файл подгрузился верно, однако есть один нюанс. Давайте посмотрим на информацию о столбцах с помощью метода info():
#melb_df.info()
#Преобразуйте столбец Date в формат datetime и выделите квартал (quarter) продажи объектов недвижимости. 
#Найдите второй по популярности квартал продажи.В качестве ответа запишите число объектов,проданных в этом квартале.
melb_df['Date'] = pd.to_datetime(melb_df['Date'])

def quarter_separator (date):
    if 0 <= date < 4:
        return '1 quarter'
    if 4 <= date < 7:
        return '2 quarter'
    if 7 <= date < 10:
        return '3 quarter'
    if 10 <= date <= 12:
        return '4 quarter'
    else:
        return 'else'

melb_df['QuarterTime'] = melb_df['Date'].dt.month.apply(quarter_separator)
print (melb_df['QuarterTime'].value_counts())
#Преобразуйте все столбцы, в которых меньше 150 уникальных значений, в тип данных category, 
# исключив из преобразования столбцы Date, Rooms, Bedroom, Bathroom, Car.
#В качестве ответа запишите результирующее количество столбцов, которые имеют тип данных category.
melb_df.nunique()
col_exclude = ['Date','Rooms','Bedroom','Bathroom','Car']
for col in melb_df.columns:
    #print (melb_df[col].nunique())
    if melb_df[col].nunique() < 150 and col not in col_exclude:
        melb_df[col] = melb_df[col].astype('category')
    #    print (melb_df[col])
melb_df.info()

3 quarter    4873
2 quarter    4359
4 quarter    2329
1 quarter    2019
Name: QuarterTime, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 27 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Suburb           13580 non-null  category      
 1   Rooms            13580 non-null  int64         
 2   Type             13580 non-null  category      
 3   Price            13580 non-null  float64       
 4   Method           13580 non-null  category      
 5   SellerG          13580 non-null  category      
 6   Date             13580 non-null  datetime64[ns]
 7   Distance         13580 non-null  float64       
 8   Postcode         13580 non-null  int64         
 9   Bedroom          13580 non-null  int64         
 10  Bathroom         13580 non-null  int64         
 11  Car              13580 non-null  int64         
 12  Landsize         13580 non-null  float64

In [4]:
"""СОРТИРОВКА ПО ЗНАЧЕНИЯМ ОДНОГО СТОЛБЦА
Приведём несколько примеров сортировки нашей таблицы с недвижимостью.
Отсортируем таблицу по возрастанию цены объектов недвижимости (Price):"""
melb_df.sort_values(by='Price').head(10)
#А теперь отсортируем таблицу по убыванию (от самой последней до самой первой) даты продажи объекта (Date). 
#Для этого выставим параметр ascending на False:
melb_df.sort_values(by='Date',ascending=False)
#Так, например, отсортируем таблицу сначала по возрастанию расстояния от центра города (Distance), 
# а затем — по возрастанию цены объекта (Price). Для того чтобы вывод был более наглядным, 
# выделим каждую десятую строку из столбцов Distance и Price результирующей таблицы:
melb_df.sort_values(by=['Price','Distance']).loc[::10,['Price','Distance']]
"""А теперь рассмотрим применение сортировки на практике.
Предположим, компания McGrath поручила нам восстановить хронологию продаж таунхаусов, у которых площадь участка 
существенно больше площади здания, чтобы понять, как часто компания справляется с таким сложным видом объектов. 
Объекты, проданные в один и тот же день, мы бы хотели сортировать по значению коэффициента соотношения площадей. 

Найдём информацию о таунхаусах (Type), проданных компанией (SellerG) McGrath, у которых коэффициент соотношения 
площадей здания и участка (AreaRatio) меньше -0.8. Результат отсортируем по дате продажи (Date) в порядке 
возрастания, а после проведём сортировку по убыванию коэффициента соотношения площадей. 
Также обновим старые индексы на новые, установив параметр ignore_index на True. 
Для наглядности результата выберем из таблицы только столбцы Data и AreaRatio:"""
mask1 = melb_df['AreaRatio'] < -0.8
mask2 = melb_df['Type'] == 'townhouse'
mask3 = melb_df['SellerG'] == 'McGrath'
melb_df[mask1 & mask2 & mask3].sort_values(
    by=['Date','AreaRatio'],
    ascending=[True, False],
    ignore_index=True
).loc[:,['Date','AreaRatio']]
#Произведите сортировку столбца AreaRatio по убыванию. При этом индексы полученной таблицы замените на новые. 
# Какое значение площади здания находится в строке 1558? Ответ округлите до целого числа.
melb_df.sort_values(by='AreaRatio',ascending=False,ignore_index=True)
print (round (melb_df['BuildingArea'].loc[1558]))
#Найдите таунхаусы (Type) с количеством жилых комнат (Rooms) больше 2. Отсортируйте полученную таблицу сначала 
#по возрастанию числа комнат, а затем по убыванию средней площади комнат (MeanRoomsSquare). 
# Индексы таблицы замените на новые. Какая цена будет у объекта в строке 18? Ответ запишите в виде целого числа.
townhouse_type = melb_df['Type'] == 'townhouse'
rooms_type = melb_df['Rooms'] > 2
melb_df[townhouse_type & rooms_type].sort_values(
    by=['Rooms','MeanRoomsSquare'],
    ascending=[True,False],
    ignore_index=True
).loc[18]
#townhouse_type = melb_df['Type'] == 'townhouse'
#melb_df.loc[townhouse_type].head(4)

126


Suburb                     Brighton East
Rooms                                  3
Type                           townhouse
Price                          1300000.0
Method                                PI
SellerG                           Buxton
Date                 2016-09-17 00:00:00
Distance                            10.7
Postcode                            3187
Bedroom                                3
Bathroom                               2
Car                                    2
Landsize                           337.0
BuildingArea                       222.0
CouncilArea                      Bayside
Lattitude                       -37.9196
Longtitude                      145.0205
Regionname         Southern Metropolitan
Propertycount                       6938
MeanRoomsSquare                    27.75
AreaRatio                      -0.205725
MonthSale                              9
AgeBuilding                           16
WeekdaySale                            5
StreetType      

In [5]:
#Применим агрегирующую функцию среднего к результату работы groupby(). 
#В качестве столбца для группировки возьмём столбец типа объекта недвижимости (Type). Если мы хотим видеть тип 
#объекта в качестве отдельного столбца таблицы, мы можем выставить параметр as_index на False:
melb_df.groupby(by='Type',as_index=False).mean()
#Как правило, нам не нужна информация обо всех столбцах, поэтому агрегирующие методы можно применять только 
#к интересующему нас столбцу. Например, давайте сравним средние цены на объекты в зависимости от их типа:
melb_df.groupby('Type')['Price'].mean()
#Теперь давайте выясним, какие регионы (Regionname) наиболее удалены от центра Мельбурна.Для этого найдём минимальное
#значение расстояния от центра города до объекта в зависимости от его региона.Результат отсортируем по убыванию расстояния:
melb_df.groupby('Regionname')['Distance'].min().sort_values(ascending=False)
#Чтобы рассчитать несколько агрегирующих методов, можно воспользоваться методом agg(), 
#который принимает список строк с названиями агрегаций.
#Давайте построим таблицу для анализа продаж по месяцам. Для этого найдём количество продаж, а также среднее 
#и максимальное значения цен объектов недвижимости (Price), сгруппированных по номеру месяца продажи (MonthSale).
#Результат отсортируем по количеству продаж в порядке убывания:
melb_df.groupby('MonthSale')['Price'].agg(
    ['count','mean','max']
).sort_values(by='count',ascending=False)
#Если вам нужна полная информация обо всех основных статистических характеристиках внутри каждой группы, 
# вы можете воспользоваться методом agg(), передав в качестве его параметра строку 'describe':
melb_df.groupby('MonthSale')['Price'].agg('describe')
"""После базовых математических функций наиболее частым агрегированием является подсчёт числа уникальных значений.
Так, например, мы можем вычислить число уникальных риелторских компаний в зависимости от региона, чтобы понять, 
в каких регионах конкуренция на рынке недвижимости меньше. Это можно сделать, передав в параметр метода agg() 
строку 'nunique'. 
Более того, метод agg() поддерживает использование и других функций. Передадим дополнительно встроенную функцию set,
чтобы получить множество из агентств недвижимости, которые работают в каждом из регионов:"""
melb_df.groupby('Regionname')['SellerG'].agg(['nunique',set])


#Сгруппируйте данные по признаку количества комнат и найдите среднюю цену объектов недвижимости в каждой группе. 
# В качестве ответа запишите количество комнат, для которых средняя цена наибольшая.
melb_df.groupby('Rooms')['Price'].mean().max()  #melb_df[melb_df['Rooms'] == 7]['Price'].mean().max()
#Какой регион имеет наименьшее стандартное отклонение по географической широте (Lattitude)?
#В качестве ответа запишите название этого региона.
melb_df.groupby('Regionname')['Lattitude'].std().sort_values()
#Какая риелторская компания (SellerG) имеет наименьшую общую выручку за период с 1 мая по 1 сентября 
#(включительно) 2017 года? Для ответа на этот вопрос рассчитайте сумму продаж (Price) каждой компании 
#в заданный период. Не забудьте перевести даты в формат datetime.
date1 = pd.to_datetime('2017-05-01')
date2 = pd.to_datetime('2017-09-01')
mask = (date1 <= melb_df['Date']) & (melb_df['Date']<= date2)
melb_df.loc[mask,:].groupby('SellerG')['Price'].sum().sort_values(ascending=True)


C:\Users\управ.ПК\AppData\Local\Temp\ipykernel_6164\3808855896.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  melb_df.groupby(by='Type',as_index=False).mean()


SellerG
LITTLE             2742000.0
Cayzer             4439000.0
Burnham            4550500.0
Moonee             7328000.0
Thomson            8332000.0
Bells              8656000.0
Alexkarbon        10985000.0
McDonald          14637500.0
Rendina           15422276.0
Nick              16890000.0
Douglas           18341000.0
Buckingham        19033000.0
C21               19515000.0
Eview             19791500.0
Collins           20217000.0
Philip            22051800.0
Chisholm          23225000.0
Williams          23297000.0
Love              23365500.0
Purplebricks      23401000.0
O'Brien           23855508.0
HAR               25568000.0
Village           26473000.0
RW                29261000.0
Raine             30687700.0
Stockdale         35409800.0
Sweeney           36882750.0
Gary              39138400.0
Hodges            43231000.0
YPA               46354350.0
Miles             47582000.0
Kay               48569500.0
RT                50498000.0
Brad              55955000.0
Jas   

In [12]:
"""МЕТОД GROUPBY КАК СПОСОБ ПОСТРОЕНИЯ СВОДНЫХ ТАБЛИЦ
На самом деле мы с вами уже строили простейшие одномерные сводные таблицы с помощью метода 
groupby — мы рассматривали сводную таблицу в контексте группировки по одному признаку. """
#Например, мы уже умеем строить таблицу, которая показывает зависимость медианной цены 
#и площади здания от числа комнат:
melb_df.groupby('Rooms')[['Price','BuildingArea']].median()
#Также можно построить таблицу, в которой мы будем учитывать не только число комнат, но и тип 
#здания (Type).Для этого в параметрах метода groupby() укажем список из нескольких интересующих 
#нас столбцов.
melb_df.groupby(['Rooms','Type'])['Price'].mean()
#Для того, чтобы финальный результат был представлен в виде сводной таблицы (первый группировочный
#признак по строкам, а второй — по столбцам), а не в виде Series с иерархическими индексами, к 
#результату чаще всего применяют метод unstack(), который позволяет переопределить вложенный 
#индекс в виде столбцов таблицы:
melb_df.groupby(['Rooms','Type'])['Price'].mean().unstack()
"""Основные параметры метода pivot_table()
values — имя столбца, по которому необходимо получить сводные данные, применяя агрегирующую функцию;
index — имя столбца, значения которого станут строками сводной таблицы;
columns — имя столбца, значения которого станут столбцами сводной таблицы;
aggfunc — имя или список имён агрегирующих функций (по умолчанию — подсчёт среднего, 'mean');
fill_value — значение, которым необходимо заполнить пропуски (по умолчанию пропуски не заполняются).
Давайте построим ту же самую таблицу, но уже с использованием метода pivot_table. В качестве
параметра values укажем столбец Price, в качестве индексов сводной таблицы возьмём Rooms, 
а в качестве столбцов — Type. Агрегирующую функцию оставим по умолчанию (среднее). 
Дополнительно заменим пропуски в таблице на значение 0. Финальный результат для наглядности 
вывода округлим с помощью метода round() до целых."""
melb_df.pivot_table(
    values='Price',
    index='Rooms',
    columns='Type',
    fill_value=0
).round()
"""А теперь давайте проанализируем продажи в каждом из регионов в зависимости от того, 
будний был день или выходной. Для этого построим сводную таблицу, в которой строками будут
являться названия регионов (Regionname), а в столбцах будет располагаться наш «признак-мигалка»
выходного дня (Weekend), который равен 1, если день был выходным, и 0 — в противном случае. 
В качестве значений сводной таблицы возьмём количество продаж."""
melb_df.pivot_table(
    values='Price',
    index='Regionname',
    columns='Weekend',
    aggfunc='count'
)
"""Разберём ещё один пример: найдём, как зависит средняя и медианная площадь участка (Landsize) 
от типа объекта (Type) и его региона (Regionname). Чтобы посмотреть несколько статистических 
параметров, нужно передать в аргумент aggfunc список из агрегирующих функций. Построим такую 
сводную таблицу, где пропущенные значения заменим на 0:"""
melb_df.pivot_table(
    values='Landsize',
    index='Regionname',
    columns='Type',
    aggfunc=['median','mean'],
    fill_value=0
)
"""МНОГОМЕРНЫЕ СВОДНЫЕ ТАБЛИЦЫ
До этого мы рассматривали, как некоторый статистический показатель может зависеть от двух признаков.
Однако, как уже упоминалось, сводные таблицы позволяют наблюдать зависимость и от большего числа
признаков. Такие сводные таблицы называются многомерными.
Давайте построим таблицу, в которой по индексам будут располагаться признаки метода продажи (Method)
и типа объекта (Type), по столбцам — наименование региона (Regionname), а на пересечении строк и 
столбцов будет стоять медианная цена объекта (Price):"""
melb_df.pivot_table(
    values='Price',
    index=['Method','Type'],
    columns='Regionname',
    aggfunc='median',
    fill_value=0
)
"""ДОСТУП К ДАННЫМ В СВОДНОЙ ТАБЛИЦЕ"""
#Запишем сводную таблицу, которую мы создавали ранее в переменную pivot:
pivot = melb_df.pivot_table(
    values='Landsize',
    index='Regionname',
    columns='Type',
    aggfunc=['median','mean'],
    fill_value=0
)
#Выведем её столбцы с помощью атрибута columns:
pivot.columns
"""В результате мы получаем объект MultiIndex. Этот объект хранит в себе шесть комбинаций пар 
столбцов (два статистических параметра и три типа здания), то есть есть шесть возможных вариантов
обращения к столбцам таблицы.
Мультииндексы раскрываются подобно вложенным словарям — по очереди, как матрёшка. Чтобы получить доступ 
к определённому столбцу, вы должны сначала обратиться к столбцу, который находится уровнем выше."""
#Так, из таблицы pivot мы можем получить средние значения площадей участков для типа здания unit,
#displaпросто последовательно обратившись по имени столбцов:
display(pivot['mean'],['unit'])
#Аналогично производится и фильтрация данных. Например, если нам нужны регионы, в которых средняя 
#площадь здания для домов типа house меньше их медианной площади, то мы можем найти их так:
mask = pivot['mean']['house'] < pivot ['median']['house']
filtred_pivot = pivot[mask]
display(filtred_pivot)
#Чтобы получить индексы отфильтрованной таблицы, можно воспользоваться атрибутом index
#и обернуть результат в список:
print (list(filtred_pivot.index))
#На самом деле мультииндексные таблицы можно создавать и вручную.
#Давайте посмотрим на синтаксис данной конструкции:
import numpy as np
mser = pd.Series (
    np.random.rand(8),
    index=[['white','white','white','blue','blue','red','red','red'], 
           ['up','down','right','up','down','up','down','left']]
)
display(mser)
"""В данном примере мы создаём объект Series со вложенными индексами. Мы передаём в качестве 
индексов Series вложенный список, где первый список задаёт внешний уровень вложенности, а второй 
список — внутренний уровень вложенности. Значения Series — случайные числа от 0 до 1, 
сгенерированные функцией np.random.rand() (ваши значения могут отличаться).
Если посмотреть на индексы Series, можно увидеть, что они являются мультииндексами:"""
print(mser.index)
#Аналогично создаются DataFrame со вложенными признаками (вложенными столбцами) — для этого 
#вложенный список передаётся в параметр columns при инициализации таблицы:
mframe = pd.DataFrame(
    np.random.randn(16).reshape(4,4),
    index=[['white','white','red','red'],['up','down','up','down']],
    columns=[['pen','pen','paper','paper'],[1,2,1,2]]
)
display(mframe)

Type,house,townhouse,unit
Regionname,,,
Eastern Metropolitan,717.422847,269.440678,330.444444
Eastern Victoria,3108.960000,0.000000,295.333333
Northern Metropolitan,619.249092,317.325733,495.026538
Northern Victoria,3355.463415,0.000000,0.000000
South-Eastern Metropolitan,664.306701,212.160000,357.864865
Southern Metropolitan,569.643881,278.858824,466.380245
Western Metropolitan,507.883406,244.560669,557.637232
Western Victoria,655.500000,0.000000,0.000000


['unit']

median                       mean              \
Type                   house townhouse unit       house   townhouse   
Regionname                                                            
Southern Metropolitan  586.0     246.0    0  569.643881  278.858824   
Western Metropolitan   531.0     198.0   62  507.883406  244.560669   

                                   
Type                         unit  
Regionname                         
Southern Metropolitan  466.380245  
Western Metropolitan   557.637232

['Southern Metropolitan', 'Western Metropolitan']


white  up       0.300357
       down     0.412963
       right    0.078306
blue   up       0.197050
       down     0.938165
red    up       0.166923
       down     0.061432
       left     0.546317
dtype: float64

MultiIndex([('white',    'up'),
            ('white',  'down'),
            ('white', 'right'),
            ( 'blue',    'up'),
            ( 'blue',  'down'),
            (  'red',    'up'),
            (  'red',  'down'),
            (  'red',  'left')],
           )


pen               paper          
                   1         2         1         2
white up    0.044455  0.695287  1.457748 -0.159748
      down -0.372204 -0.187931 -0.595188  0.199505
red   up    0.217343 -0.177095 -0.594923 -0.697102
      down -1.036062  0.104137  0.319674 -0.775116

In [17]:
#Составьте сводную таблицу, которая показывает зависимость медианной площади (BuildingArea) здания
#от типа объекта недвижимости (Type) и количества жилых комнат в доме (Rooms). 
# Для какой комбинации признаков площадь здания наибольшая?
#В качестве ответа запишите эту комбинацию (тип здания, число комнат) через запятую, без пробелов.
melb_df.groupby('BuildingArea')[['Type','Rooms']].median()
pivot = melb_df.pivot_table(
    values='BuildingArea',
    index='Type',
    columns='Rooms',
    aggfunc='median',
    fill_value=0
)
display(pivot)
#Составьте сводную таблицу, которая показывает зависимость медианной цены объекта недвижимости 
#(Price) от риелторского агентства (SellerG) и типа здания (Type).
#Во вновь созданной таблице найдите агентство, у которого медианная цена для зданий типа unit 
#максимальна. В качестве ответа запишите название этого агентства.
pivot = melb_df.pivot_table(
    values='Price',
    index='SellerG',
    columns='Type',
    aggfunc='median',
    fill_value=0
)
max_unit_price = max (pivot['unit'])
print (max_unit_price)
print(pivot[pivot['unit'] == max_unit_price].index[0])#срез по условию
#max_unit_price = pivot['unit'].max()
#print (max_unit_price)




C:\Users\управ.ПК\AppData\Local\Temp\ipykernel_8396\3108749247.py:5: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  melb_df.groupby('BuildingArea')[['Type','Rooms']].median()


Rooms,1,2,3,4,5,6,7,8,10
Type,,,,,,,,,
house,126.0,126,126,141.0,177,126,216.5,126,126
townhouse,88.0,114,126,159.5,152,0,0.0,0,0
unit,69.5,110,126,126.0,0,171,0.0,126,0


900000
Nick


In [9]:
melb_df.groupby('Type')[['Rooms','BuildingArea']].agg({'Rooms':['max','min'],'BuildingArea':'median'})

Rooms     BuildingArea
            max min       median
Type                            
house        10   1        126.0
townhouse     5   1        126.0
unit          8   1        126.0